In [61]:
import pandas as pd
import numpy as np
import plotly.express as px

# Import data

In [62]:
sp500 = pd.read_excel('data/s&p500.xlsx')
sp500_top10_holders = pd.read_excel('data/sp500_top10_holders.xlsx')

In [63]:
# Fix index and column names
sp500.columns = sp500.iloc[0]
sp500 = sp500[1:]

sp500_top10_holders.columns = sp500_top10_holders.iloc[0]
sp500_top10_holders = sp500_top10_holders[1:]

In [ ]:
sp500.head()

## Clean shareholders table

In [ ]:
# Delete rows 'Top mutual fund holders' in 'class' column
sp500_top10_holders = sp500_top10_holders[sp500_top10_holders['class'] != 'Top mutual fund Holders']
sp500_top10_holders.shape

# Remove duplicated rows
sp500_top10_holders = sp500_top10_holders.drop_duplicates()
sp500_top10_holders.shape

## Merge Data

In [ ]:
merged_df = pd.merge(sp500_top10_holders, sp500, on=['symbol', 'isin'])
merged_df.head()

In [ ]:
# merged_df[['isin', 'symbol']].nunique()
merged_df.shape


## Companies not located on US

In [ ]:
merged_df['country'].unique()

## Convert values

In [70]:
# Function to convert values with 'B' and 'M' suffixes to numeric
def convert_shares(value):
    if 'B' in value:
        return float(value.replace('B', '')) * 1e9
    elif 'M' in value:
        return float(value.replace('M', '')) * 1e6
    elif 'k' in value:
        return float(value.replace('k', '')) * 1e3
    else:
        return float(value)

# Apply the function to the 'shares' column
merged_df['shares'] = merged_df['shares'].apply(convert_shares)

In [ ]:
# List of columns to convert to numeric
columns_to_numeric = [
    '%', 'value', 'marketCap', 'enterpriseValue', 'totalCash', 'totalCashPerShare',
    'ebitda', 'totalDebt', 'quickRatio', 'currentRatio', 'totalRevenue', 'debtToEquity',
    'revenuePerShare', 'returnOnAssets', 'returnOnEquity', 'freeCashflow', 'operatingCashflow',
    'earningsGrowth', 'revenueGrowth', 'grossMargins', 'ebitdaMargins', 'operatingMargins'
]

# Convert specified columns to numeric
merged_df[columns_to_numeric] = merged_df[columns_to_numeric].apply(pd.to_numeric, errors='coerce')

# Display the data types to verify the changes
merged_df.head()


# Treemap

## Company -> Shareholder -> Details

In [ ]:
# Create a tree map using plotly.express
fig = px.treemap(
    merged_df, 
    path=['name', 'symbol'],
    values='value',
    color='value', 
    hover_data=['shares'],
    color_continuous_scale='RdBu',
    range_color=[0, merged_df['value'].max()],
    color_continuous_midpoint=np.average(merged_df['value'], weights=merged_df['shares'])
)

# Change the figure size
fig.update_layout(
    width=1000,
    height=800
)
fig.show()

# Circular Packing (?)